### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [8]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [9]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model

In [10]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [11]:
# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [ ]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/Users/revathithangadurai/miniconda3/envs/ats_env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/revathithangadurai/miniconda3/envs/ats_env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/revathithangadurai/miniconda3/envs/ats_env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/revathithangadurai/miniconda3/envs/ats_env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a futu

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50


2025-08-30 00:13:54.015162: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-08-30 00:13:54.020610: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-08-30 00:13:54.037202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-08-30 00:13:54.050090: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-08-30 00:13:54.050639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-08-30 00:13:54.090487: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


Epoch 1/50


...

 41/167 ━━━━━━━━━━━━━━━━━━━━ 14s 114ms/step - accuracy: 0.7832 - loss: 0.5264Epoch 1/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 17s 100ms/step - accuracy: 0.7955 - loss: 0.5037
 93/167 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - accuracy: 0.8085 - loss: 0.4601Epoch 50/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step - accuracy: 0.7900 - loss: 0.5359
Epoch 48/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 16s 97ms/step - accuracy: 0.8050 - loss: 0.4477
Epoch 11/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 15s 91ms/step - accuracy: 0.8035 - loss: 0.5054
Epoch 11/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 17s 99ms/step - accuracy: 0.7984 - loss: 0.4950
 47/167 ━━━━━━━━━━━━━━━━━━━━ 13s 112ms/step - accuracy: 0.8179 - loss: 0.4516Epoch 11/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 16s 96ms/step - accuracy: 0.8091 - loss: 0.4527
Epoch 12/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 20s 115ms/step - accuracy: 0.8013 - loss: 0.4719
 95/167 ━━━━━━━━━━━━━━━━━━━━ 8s 113ms/step - accuracy: 0.8096 - loss: 0.4595

/Users/revathithangadurai/miniconda3/envs/ats_env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/revathithangadurai/miniconda3/envs/ats_env/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


167/167 ━━━━━━━━━━━━━━━━━━━━ 19s 113ms/step - accuracy: 0.7819 - loss: 0.5983
Epoch 49/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 28s 128ms/step - accuracy: 0.7814 - loss: 0.4837
Epoch 2/50
149/167 ━━━━━━━━━━━━━━━━━━━━ 2s 113ms/step - accuracy: 0.8124 - loss: 0.4844Epoch 1/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 19s 116ms/step - accuracy: 0.8086 - loss: 0.4463
Epoch 12/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 20s 119ms/step - accuracy: 0.8020 - loss: 0.5145
Epoch 12/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 20s 118ms/step - accuracy: 0.7997 - loss: 0.4916
 13/167 ━━━━━━━━━━━━━━━━━━━━ 12s 82ms/step - accuracy: 0.7906 - loss: 0.5114Epoch 12/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 32s 140ms/step - accuracy: 0.7829 - loss: 0.4712
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 19s 113ms/step - accuracy: 0.8078 - loss: 0.4573
Epoch 13/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 17s 101ms/step - accuracy: 0.7825 - loss: 0.5632
Epoch 50/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.8054 - loss: 0.4395
 62/167 ━━━━━━━━━━━━━━━━━━━━ 11s 109ms/st

/Users/revathithangadurai/miniconda3/envs/ats_env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/revathithangadurai/miniconda3/envs/ats_env/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


167/167 ━━━━━━━━━━━━━━━━━━━━ 16s 91ms/step - accuracy: 0.8065 - loss: 0.4617
Epoch 14/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 16s 91ms/step - accuracy: 0.8046 - loss: 0.4408
101/167 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - accuracy: 0.8099 - loss: 0.4655Epoch 4/50
145/167 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.7898 - loss: 0.54219Epoch 1/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 15s 86ms/step - accuracy: 0.7896 - loss: 0.5446
Epoch 14/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 15s 87ms/step - accuracy: 0.7883 - loss: 0.5123
 12/167 ━━━━━━━━━━━━━━━━━━━━ 11s 77ms/step - accuracy: 0.7366 - loss: 0.7055Epoch 14/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step - accuracy: 0.8102 - loss: 0.4472
Epoch 4/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 14s 81ms/step - accuracy: 0.8003 - loss: 0.4675
 20/167 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - accuracy: 0.8121 - loss: 0.4145Epoch 15/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 14s 84ms/step - accuracy: 0.8027 - loss: 0.4548
Epoch 3/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 13s 74ms/step - accuracy: 0.8065 - lo